In [1]:
\cd /home/qenv/

In [1]:
\l model/init.q

[0;31mmodel/init.q. OS reports: No such file or directory[0m: [0;31mmodel/init.q. OS reports: No such file or directory[0m

In [2]:
\ls

"Dockerfile"
"README.md"
"TODO.md"
"__init__.py"
"atari.sh"
"binance.q"
"common"
"config"
"engine"
"env.q"
"envTest.q"
"ingest"
"kdb.sh"
"kdbj.sh"
"notebooks"
"pipeline.q"
"qenv_cluster.sh"
"quantest"
"research"
"server.q"
"serverTest.q"
"state"
..
